# RAG 와 LLM 을 활용한 질의 응답 실습

 - LLM API 호출 시 RAG를 통해 연관 정보 추출 후 Prompt 에 추가하는 실습
 - RAG 추가 전후로 평가 진행

In [1]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [2]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

### 평가
- RAGAS의 Answer Correctedness 평가 지표 end to end
  - 모델의 답변과 실제 정답 간의 일치율 측정

In [3]:
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [01:09<00:00,  6.94s/it]


In [12]:
import os

from dotenv import load_dotenv
from openai import OpenAI
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import answer_correctness

load_dotenv()
MY_OPENAI_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(api_key=MY_OPENAI_KEY)

data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness], llm=llm)
score

AttributeError: 'OpenAI' object has no attribute 'set_run_config'

In [13]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [14]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 복합적으로 작용한 결과입니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **중앙은행의 통화정책**: 글로벌 금융위기 이후 많은 국가의 중앙은행들이 경제 회복을 촉진하기 위해 금리를 낮추고 양적 완화 정책을 시행했습니다. 이러한 정책들은 시장에 유동성을 공급하고 금리를 낮추는 효과를 가져왔습니다.

2. **저성장 및 저물가**: 선진국을 중심으로 경제 성장률이 둔화되고 물가 상승률이 낮아지면서 중앙은행들은 금리를 낮게 유지할 필요성을 느꼈습니다. 저성장과 저물가는 금리를 낮추는 주요 요인 중 하나입니다.

3. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 소비보다는 저축이 증가하는 경향이 있습니다. 이는 자본 공급을 증가시키고 금리를 낮추는 요인으로 작용합니다.

4. **글로벌 불확실성**: 무역 갈등, 지정학적 리스크, 코로나19 팬데믹 등 글로벌 불확실성이 증가하면서 안전자산에 대한 수요가 높아졌습니다. 이는 국채와 같은 안전자산의 금리를 낮추는 결과를 초래했습니다.

5. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고 비용이 절감되면서 물가 상승 압력이 줄어들었습니다. 이는 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성했습니다.

6. **부채 증가**: 많은 국가들이 높은 부채 수준을 유지하고 있으며, 이는 금리를 낮게 유지해야 하는 압력으로 작용합니다. 높은 금리는 부채 상환 부담을 증가시키기 때문에 정부와 기업들은 낮은 금리를 선호합니다.

이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.


#### LLM + RAG
- text-embedding-3-large 임베딩 모델 사용

In [ ]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    most_relevant_index = np.argmax(similarities)
    
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

In [ ]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_relevancy, context_recall, context_precision

data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

In [ ]:
print(predictions[0])

In [ ]:
print(answers[0])